In [1]:
import pandas as pd

In [2]:
path = "D:\Engineering Courses\Data Science\Projects\Data Science Project From Scratch\ds_salary_proj-master\glassdoor_jobs.csv"
df = pd.read_csv(path)

In [3]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [4]:
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [5]:
df.shape

(956, 15)

In [6]:
df.columns

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

## Remving rows with -1 salary value

In [7]:
df = df[df['Salary Estimate'] != '-1']

## Removing unnamed column

In [8]:
df.drop('Unnamed: 0',axis = 1,inplace = True)

In [9]:
df.shape

(742, 14)

## Salary

In [10]:
salary1 = df['Salary Estimate'].apply(lambda x:x.split('(')[0])
salary2 = salary1.apply(lambda x:x.replace('$','').replace('K',''))

In [11]:
df['hourly'] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)

In [12]:
df['employer provided'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary' in x.lower() else 0)

In [13]:
salary2 = salary2.apply(lambda x: x.lower().replace('per hour',''))

In [14]:
salary2 = salary2.apply(lambda x: x.lower().replace('employer provided salary:',''))

In [15]:
salary2.head()

0     53-91 
1    63-112 
2     80-90 
3     56-97 
4    86-143 
Name: Salary Estimate, dtype: object

In [16]:
df['min_salary'] = salary2.apply(lambda x:int(x.split('-')[0]))

In [17]:
df['max_salary'] = salary2.apply(lambda x:int(x.split('-')[1]))

In [18]:
df['min_salary'] = df.apply(lambda x:x.min_salary * 2 if x.hourly == 1 else x.min_salary , axis = 1)
df['max_salary'] = df.apply(lambda x:x.max_salary * 2 if x.hourly == 1 else x.max_salary , axis = 1)

In [19]:
df['avg_salary'] = (df['min_salary'] + df['max_salary'])/2

In [20]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'employer provided', 'min_salary', 'max_salary',
       'avg_salary'],
      dtype='object')

## Job Title

In [21]:
def Job_simple(title):
    if 'scientist' in title.lower():
        return 'data scientist'
    
    elif 'data engineer' in title.lower():
        return 'data engineer'
    
    elif 'analyst' in title.lower():
        return 'analyst'
    
    elif 'machine learning' in title.lower():
        return 'mle'
    
    elif 'manager' in title.lower():
        return 'manager'
    
    elif 'director' in title.lower():
        return 'director'
    
    else:
        return 'NA'
    

def seniority(title):
    if 'senior' in title.lower() or 'sr' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'senior'
    
    elif 'jr' in title.lower() or 'jr.' in title.lower() or 'junior' in title.lower():
        return 'jr'
    
    else:
        return 'NA'

In [22]:
df['Job_simp'] = df['Job Title'].apply(Job_simple)
df['Job_simp'].value_counts()

data scientist    438
data engineer     119
analyst           100
NA                 41
manager            21
mle                12
director           11
Name: Job_simp, dtype: int64

In [23]:
df['Seniority'] = df['Job Title'].apply(seniority)
df['Seniority'].value_counts()

NA        519
senior    220
jr          3
Name: Seniority, dtype: int64

## Company Name

In [24]:
df['company_txt'] = df.apply(lambda x:x['Company Name'][:-3] if int(x['Rating']) > 0  else x['Company Name'],axis = 1)

In [25]:
df['company_txt'] = df['company_txt'].apply(lambda x:x.replace('\n',''))

In [26]:
df['company_txt'].head()

0                        Tecolote Research
1    University of Maryland Medical System
2                                  KnowBe4
3                                     PNNL
4                       Affinity Solutions
Name: company_txt, dtype: object

## Founded

In [27]:
df['Age'] = df['Founded'].apply(lambda x:2020-int(x))

## Job Description

In [28]:
df['job_desc_len'] = df['Job Description'].apply(lambda x:len(x.split(',')))

In [29]:
df['Python'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
df['Spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
df['Excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
df['AWS'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)

In [30]:
df['Python'].value_counts()

1    392
0    350
Name: Python, dtype: int64

In [31]:
df['Spark'].value_counts()

0    575
1    167
Name: Spark, dtype: int64

In [32]:
df['Excel'].value_counts()

1    388
0    354
Name: Excel, dtype: int64

In [33]:
df['AWS'].value_counts()

0    566
1    176
Name: AWS, dtype: int64

## Location

In [34]:
df['State'] = df['Location'].apply(lambda x: x.split(',')[1])

In [35]:
df['Same State'] = df.apply(lambda x: 1 if x['Location'] == x['Headquarters'] else 0 ,axis = 1)

In [36]:
df['State'] = df['State'].apply(lambda x: 'CA' if x.lower() == 'los angeles' else x)

In [37]:
df['State'].value_counts()

 CA             151
 MA             103
 NY              72
 VA              41
 IL              40
 MD              35
 PA              33
 TX              28
 NC              21
 WA              21
 NJ              17
 FL              16
 OH              14
 TN              13
 CO              11
 DC              11
 UT              10
 WI              10
 IN              10
 AZ               9
 MO               9
 AL               8
 DE               6
 GA               6
 KY               6
 MI               6
 IA               5
 CT               5
 OR               4
 NE               4
 LA               4
 NM               3
 KS               3
 MN               2
 ID               2
 Los Angeles      1
 RI               1
 SC               1
Name: State, dtype: int64

## Competitors

In [38]:
df['num_comp'] = df['Competitors'].apply(lambda x:len(x.split(',')) if x != '-1' else 0)

In [39]:
df.to_csv('salary_data_cleaned.csv')

In [40]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,hourly,employer provided,min_salary,max_salary,avg_salary,Job_simp,Seniority,company_txt,Age,job_desc_len,Python,Spark,Excel,AWS,State,Same State,num_comp
0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1,0,0,53,91,72.0,data scientist,NA,Tecolote Research,47,38,1,0,1,0,NM,0,0
1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1,0,0,63,112,87.5,data scientist,NA,University of Maryland Medical System,36,48,1,0,0,0,MD,0,0
2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1,0,0,80,90,85.0,data scientist,NA,KnowBe4,10,34,1,1,1,0,FL,1,0
3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa...",0,0,56,97,76.5,data scientist,NA,PNNL,55,49,1,0,0,0,WA,1,3
4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee",0,0,86,143,114.5,data scientist,NA,Affinity Solutions,22,28,1,0,1,0,NY,1,3
5,Data Scientist,$71K-$119K (Glassdoor est.),CyrusOne is seeking a talented Data Scientist ...,3.4,CyrusOne\n3.4,"Dallas, TX","Dallas, TX",201 to 500 employees,2000,Company - Public,Real Estate,Real Estate,$1 to $2 billion (USD),"Digital Realty, CoreSite, Equinix",0,0,71,119,95.0,data scientist,NA,CyrusOne,20,31,1,0,1,1,TX,1,3
6,Data Scientist,$54K-$93K (Glassdoor est.),Job Description\n\n**Please only local candida...,4.1,ClearOne Advantage\n4.1,"Baltimore, MD","Baltimore, MD",501 to 1000 employees,2008,Company - Private,Banks & Credit Unions,Finance,Unknown / Non-Applicable,-1,0,0,54,93,73.5,data scientist,NA,ClearOne Advantage,12,8,0,0,1,0,MD,1,0
7,Data Scientist,$86K-$142K (Glassdoor est.),Advanced Analytics – Lead Data Scientist\nOver...,3.8,Logic20/20\n3.8,"San Jose, CA","Seattle, WA",201 to 500 employees,2005,Company - Private,Consulting,Business Services,$25 to $50 million (USD),-1,0,0,86,142,114.0,data scientist,NA,Logic20/20,15,26,1,1,1,1,CA,0,0
8,Research Scientist,$38K-$84K (Glassdoor est.),SUMMARY\n\nThe Research Scientist I will be ta...,3.3,Rochester Regional Health\n3.3,"Rochester, NY","Rochester, NY",10000+ employees,2014,Hospital,Health Care Services & Hospitals,Health Care,$500 million to $1 billion (USD),-1,0,0,38,84,61.0,data scientist,NA,Rochester Regional Health,6,8,0,0,0,0,NY,1,0
9,Data Scientist,$120K-$160K (Glassdoor est.),isn’t your usual company. Our work is powered ...,4.6,<intent>\n4.6,"New York, NY","New York, NY",51 to 200 employees,2009,Company - Private,Internet,Information Technology,$100 to $500 million (USD),"Clicktripz, SmarterTravel",0,0,120,160,140.0,data scientist,NA,<intent>,11,49,1,1,0,0,NY,1,2
